In [1]:
import os
import torch
import torch.nn as nn

In [2]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project'

In [5]:
os.environ["ML_FLOW_TRACKING_URI"]="https://dagshub.com/JusticeTomlinson/MLOPS-Demo-Project.mlflow"
os.environ["ML_FLOW_TRACKING_USERNAME"]="JusticeTomlinson"
os.environ["ML_FLOW_TRACKING_PASSWORD"]="1d7b52e3c0614ff8ab57826e5bff9a1391f18bd3"

In [6]:
from dataclasses import dataclass
from pathlib import Path

#need to change according to my training needs
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    input_dim: int 
    hidden1_dim: int 
    hidden2_dim: int 
    output_dim: int
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from src.mlops_project.constants import *
from src.mlops_project.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.HeartDiseaseNN
        schema = self.schema.TARGET_COLUMN


        create_directories([config.root_dir])

        model_training_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            input_dim = params.input_dim,
            hidden1_dim = params.hidden1_dim,
            hidden2_dim = params.hidden2_dim,
            output_dim = params.output_dim,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/JusticeTomlinson/MLOPS-Demo-Project.mlflow"
        )
        
        return model_training_config

In [9]:
import os
from src.mlops_project import logger
import pandas as pd
import mlflow
import mlflow.sklearn
import numpy as np
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch
from torch.utils.data import DataLoader, TensorDataset
from mlflow.models import infer_signature
import dagshub

from src.mlops_project.utils.model_architecture import NeuralNetwork

In [10]:
def save_json(path, data):
    import json
    with open(path, 'w') as f:
        # Convert NumPy types to Python native types explicitly
        converted_data = {k: (v.item() if isinstance(v, np.generic) else v) for k, v in data.items()}
        json.dump(converted_data, f)

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config=config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_squared_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    
    def log_into_mlflow(self):
        # Load the test data

        test_data = pd.read_csv(self.config.test_data_path)
        test_data = test_data.dropna()

        # Load the PyTorch model (assuming it's saved with torch.save)
        model = NeuralNetwork(self.config.input_dim,
                               self.config.hidden1_dim, 
                               self.config.hidden2_dim, 
                               self.config.output_dim)
        
        model.load_state_dict(torch.load(self.config.model_path))

        model.eval()  # Set the model to evaluation mode

        # Prepare the test inputs and labels
        test_x = torch.tensor(test_data.drop([self.config.target_column], axis=1).values, dtype=torch.float32)
        test_y = torch.tensor(test_data[[self.config.target_column]].values, dtype=torch.float32)
        dagshub.init(repo_name="MLOPS-Demo-Project", repo_owner="JusticeTomlinson")


        # Set up MLflow
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Make predictions
            with torch.no_grad():  # Disable gradient calculation
                predicted_qualities = model(test_x)

            # Convert predictions to match the expected format
            predicted_qualities = predicted_qualities.numpy().flatten()

            # Evaluate metrics
            (rmse, mae, r2) = self.eval_metrics(test_y.numpy(), predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Handle model logging and registration

            if tracking_url_type_store != "file":
                # mlflow.pytorch.log_model(model, "model", registered_model_name="HearnDiseaseNN")

                mlflow.pytorch.log_model(pytorch_model=model, 
                                         artifact_path="model",
                                         registered_model_name="NeuralNetwork")



            else:
                # mlflow.pytorch.log_model(model, "model")
                # mlflow.pytorch.log_model(model, "model", pip_requirements="pip_requirements.txt")
                mlflow.pytorch.log_model(pytorch_model=model, 
                                         artifact_path="model",
                                         registered_model_name="NeuralNetwork")


    

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-06-08 22:58:27,327: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-08 22:58:27,329: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-08 22:58:27,330: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-08 22:58:27,331: INFO: common: created directory at: artifacts]
[2024-06-08 22:58:27,333: INFO: common: created directory at: artifacts/model_evaluation]


Initialized MLflow to track repo "JusticeTomlinson/MLOPS-Demo-Project"

[2024-06-08 22:58:27,738: INFO: helpers: Initialized MLflow to track repo "JusticeTomlinson/MLOPS-Demo-Project"]


Repository JusticeTomlinson/MLOPS-Demo-Project initialized!

[2024-06-08 22:58:27,740: INFO: helpers: Repository JusticeTomlinson/MLOPS-Demo-Project initialized!]


2024/06/08 22:58:28 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/06/08 22:58:33 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
c:\Users\justi\Desktop\MLOps Demo\.venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'NeuralNetwork' already exists. Creatin